<a href="https://colab.research.google.com/github/dangduong2806/NMT-project/blob/main/Hu%E1%BA%A5n_luy%E1%BB%87n_tokenizer_v%C3%A0_l%C3%A0m_t%E1%BA%ADp_validate_ban_%C4%91%E1%BA%A7u.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Liên kết với google drive để tải file dữ liệu lên

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/nmt_data.csv /content/

Đọc file dữ liệu ra dataframe

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('nmt_data.csv')

In [ ]:
df

,Unnamed: 0,English,Vietnamese
0,0,it begins with a countdown.,câu chuyện bắt đầu với buổi lễ đếm ngược.
1,1,"on august 14th, 1947, a woman in bombay goes i...","ngày 14, tháng 8, năm 1947, gần nửa đêm, ở bom..."
2,2,"across india, people hold their breath for the...","cùng lúc, trên khắp đất ấn, người ta nín thở c..."
3,3,"and at the stroke of midnight, a squirming inf...","khi đồng hồ điểm thời khắc nửa đêm, một đứa tr..."
4,4,"these events form the foundation of ""midnight'...","những sự kiện này là nền móng tạo nên ""những đ..."
...,...,...,...
299996,299996,and you can even see how i go from frankfurt b...,bạn thậm chí có thể biết được tôi tới frankfur...
299997,299997,all this is possible with this information.,tất cả đều có thể với những thông tin này.
299998,299998,that is a little bit scary.,tôi đã có một chút sợ hãi.
299999,299999,but it is not only about me.,nhưng những thông tin đó không chỉ nói về tôi.


Tạo 2 file txt gồm 300 nghìn cặp câu song ngữ Anh - Việt

In [ ]:
with open("300kenglish.txt", "w", encoding="utf-8") as f:
  for i in range(0, 300001):
    f.write(df.loc[i, 'English'] + "\n")

In [ ]:
with open("300kvietnam.txt", "w", encoding="utf-8") as f:
  for i in range(0, 300001):
    f.write(df.loc[i, 'Vietnamese'] + "\n")

Huấn luyện BPE (Byte Pair Encoding)

In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, decoders

tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()  #Thêm decoder

trainer = trainers.BpeTrainer(
    vocab_size=16000,
    special_tokens=["<pad>", "<unk>", "<sos>", "<eos>"]
)

tokenizer.train(["300kenglish.txt"], trainer)
tokenizer.save("300kenglish_bpe_bytelevel.json")

In [ ]:
tokenizer = Tokenizer.from_file("300kenglish_bpe_bytelevel.json")

out = tokenizer.encode("apples")

print(out.tokens)
print(out.ids)
print(tokenizer.decode(out.ids))
for i in out.ids:
  print(tokenizer.decode([i]), end= "|")

['Ġapples']
[10113]
 apples
 apples|

In [ ]:
!cp /content/300kenglish_bpe_bytelevel.json /content/drive/MyDrive/

In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, decoders

tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()  #Thêm decoder

trainer = trainers.BpeTrainer(
    vocab_size=16000,
    special_tokens=["<pad>", "<unk>", "<sos>", "<eos>"]
)

tokenizer.train(["300kvietnam.txt"], trainer)
tokenizer.save("300kvietnamese_bpe_bytelevel.json")

In [ ]:
tokenizer = Tokenizer.from_file("300kvietnamese_bpe_bytelevel.json")

out = tokenizer.encode("chị dậu")

print(out.tokens)
print(out.ids)
print(tokenizer.decode(out.ids))
for i in out.ids:
  print(tokenizer.decode([i]), end= "|")

['Ġchá»ĭ', 'Ġd', 'áºŃu']
[980, 175, 938]
 chị dậu
 chị| d|ậu|

In [ ]:
!cp /content/300kvietnamese_bpe_bytelevel.json /content/drive/MyDrive/

Làm tập validate

In [ ]:
from google.colab import files
upload = files.upload()

Saving test_cleaned.csv to test_cleaned.csv


In [ ]:
df2 = pd.read_csv('test_cleaned.csv')

In [ ]:
df2

,English,Vietnamese
0,"brother albert barnett and his wife, sister su...","anh albert barnett và chị susan barnett, thuộc..."
1,severe storms ripped through parts of the sout...,"ngày 11 và 12-1-2020, những cơn bão lớn đã qué..."
2,"two days of heavy rain, high winds, and numero...",những trận mưa to và gió lớn trong suốt hai ng...
3,"sadly, brother albert barnett and his wife, si...","đáng buồn là anh albert barnett 85 tuổi, và vợ..."
4,the united states branch also reports that at ...,chi nhánh hoa kỳ cũng cho biết có ít nhất bốn ...
...,...,...
19144,"more importantly, perhaps, military rd spendin...","quan trọng hơn, có lẽ chi tiêu cho nghiên cứu ..."
19145,assuming a low average of 5 percent of overall...,giả sử một mức trung bình thấp là 5 tổng ngân ...
19146,"in fact, the pla already appears to be reaping...","thực ra, quân giải phóng nhân dân trung quốc c..."
19147,other computers had been named after daughters...,tất cả các máy tính khác đều được đặt theo tên...


Tạo file txt gồm 10000 cặp câu song ngữ Anh - Việt

In [ ]:
with open('english_validate.txt', 'w', encoding= 'utf-8') as f:
  for i in range(0, 10001):
    f.write(df2.loc[i, 'English'] + "\n")

In [ ]:
!cp /content/english_validate.txt /content/drive/MyDrive/

In [ ]:
with open('vietnam_validate.txt', 'w', encoding= 'utf-8') as f:
  for i in range(0, 10001):
    f.write(df2.loc[i, 'Vietnamese'] + "\n")

In [ ]:
!cp /content/vietnam_validate.txt /content/drive/MyDrive/